# word2vec 단어 간 유사도 체크
- 최대한 육아를 주제로 한 말뭉치를 모아보았다. (Total 2,755 documents)
- 우리가 원하는 의도에 일치하는 단어 간 유사도 결과가 나오는지를 확인하는 것이 목적이다.

## Import

In [1]:
import pandas as pd
import numpy as np
import re

from konlpy.tag import Mecab
from konlpy.tag import Komoran
from gensim.models import Word2Vec

### Datas

In [2]:
df1 = pd.read_csv("data/maeili.csv", encoding='utf-8')
df2 = pd.read_csv("data/baby.csv", encoding='utf-8')
df3 = pd.read_csv("data/toys.csv", encoding='utf-8')
df4 = pd.read_csv("data/seoulchildcare.csv", encoding='utf-8')
df5 = pd.read_csv("data/babynews_DLC.csv", encoding='utf-8')
df6 = pd.read_csv("data/ildongfoodis.csv", encoding='utf-8')
df7 = pd.read_csv("data/momsmagazine.csv", encoding='utf-8')
df8 = pd.read_csv("data/ange.csv", encoding='utf-8')
df9 = pd.read_csv("data/babytimes.csv", encoding='utf-8')
df10 = pd.read_csv("data/daekyo.csv", encoding='utf-8')
df11 = pd.read_csv("data/mom&enfant.csv", encoding='utf-8')
df12 = pd.read_csv("data/namyang.csv", encoding='utf-8')

### Concatenate Dataframe

In [3]:
df = pd.concat([df1.text, df2.text, df3.text, df4.text, df5.text,
                df6.text, df7.text, df8.text, df9.text, df10.text,
                df11.text, df12.text])
df = pd.DataFrame(df).reset_index(drop=True)
df.tail()

,text
27050,아토피에 유전적인 요인이 작용한다는 사실 알고 계시나요 유전적인 요인 뿐 아니라 아...
27051,1st 시기에 따라 필요한 영양소를 충분히 뱃속의 아이는 엄마가 먹는 것을 함께 먹...
27052,분만 여성만이 누릴 수 있는 특권 대부분의 여성들은 임신과 분만이라는 과정을 경험하...
27053,최근 전자파가 건강상의 문제를 일으킬 수 있다는 견해가 많아지면서 임신부들 또한 가...
27054,임신 기간을 설명할 때 헷갈리는 경우가 많다 책을 볼 때 의사의 설명을 들을 때 정...


### Preprocessing

In [4]:
def cleaningText(text):
    text = re.sub('&nbsp;|\xa0|\u200b|\r|\n|\t', ' ', text)
    text = re.sub('<.+>|【.+】|\[.+\]', ' ', text)
    text = re.sub('[-_=+,/\?;:^$.@*\"※~&%ㆍ!”“’「」『』〈〉\\‘|\(\)\{\}\[\]\<\>`\'…《》]', ' ', text)
    text = re.sub('[㈜×–～→▪‧•∘✽○□◇△▽▷◁☆◈◐▶▼▲◆★♦■♥️●♠🖤]', ' ', text)
    text = re.sub('[①②③④⑤⑥⑦⑧⑨⑩⑪⑫⑬⑭⑮⑯⑰⑱⑲⑳]', ' ', text)
    text = re.sub('[➊❷➌❹❺❻❼❽❾❿]', ' ', text)
    text = re.sub('http.+com|http.+co|http.+kr|www.+kr|www.+com|www.+net|http.+net', ' ', text)
    text = re.sub("\s.+\s기자", '', text)
    text = re.sub('ⓒ\S+\s|㈜\S+\s|홈페이지\S+\s|전화\S+\s|대표전화\S+\s|☎\S+\s|http\S+\s', '', text)
    
    text = re.sub('\s+', ' ', text)
    text = text.strip()

    return text


In [5]:
df = df.drop(index=[952, 955, 956, 960, 962, 964, 1165, 1240, 1322])
df = df.drop_duplicates()
df = df.dropna()
df = pd.DataFrame(df).reset_index(drop=True)

In [6]:
df['text'] = df['text'].apply(cleaningText)
df = df.dropna()
df.to_csv('data/text_for_word2vec.csv', index=False, encoding='utf-8')

In [7]:
df

,text
0,6살 남아 입니다 회사 복직 후 3살부터 주중에는 외할머니댁에서 2살 4살 터울의 ...
1,요즘 들어 아빠가 아이에게 다가가려고 하면 엄마한테 갈 거야 하면서 엄마를 유독 찾...
2,둘째가 태어나면 첫째가 힘들 거라는 건 예상하고 있었지만 툭하면 삐치고 울고 좀처럼...
3,20개월 남아가 있고 임신 8개월차로 두 아기의 엄마입니다 첫 아이는 잠자리 독립을...
4,14개월 남자 아기인데 심하게 깨무는 버릇이 있어요 전에는 졸릴 때 주로 물곤 했는...
...,...
25344,가임기 연력 여성의 흡연률 증가와 그 폐해 현대 여성은 다양한 사회활동에 참여하고 ...
25345,아토피에 유전적인 요인이 작용한다는 사실 알고 계시나요 유전적인 요인 뿐 아니라 아...
25346,1st 시기에 따라 필요한 영양소를 충분히 뱃속의 아이는 엄마가 먹는 것을 함께 먹...
25347,분만 여성만이 누릴 수 있는 특권 대부분의 여성들은 임신과 분만이라는 과정을 경험하...


### Add tokenized column

191210 정체불명의 error 발생
```
NotImplementedError: Wrong number or type of arguments for overloaded function 'Tagger_parse'.
  Possible C/C++ prototypes are:
    MeCab::Tagger::parse(MeCab::Model const &,MeCab::Lattice *)
    MeCab::Tagger::parse(MeCab::Lattice *) const
    MeCab::Tagger::parse(char const *)
```

babynews 데이터가 추가된 후 갑자기 발생한 에러인데.. 왜인지 모르겠지만 해당 부분 데이터프레임만 따로 tokenize하는 것은 문제없이 돌아간다.  
따라서 번거롭지만 분리하여 tokenizing 한 후에 합치기로 하였다.  

-> 해결.. `NaN` value 문제였다(어이없게도 NaN 값이 중간에 섞여있었음;). 누락 데이터를 잘 체크하는 습관을 갖도록하자.

In [8]:
def tokenize(text):
    tokenizer = Mecab()
    text = tokenizer.morphs(text)
#     text = ' '.join(text)
    return text

In [9]:
%%time
df['tokenized'] = df['text'].apply(tokenize)
df.tail()

CPU times: user 1min 1s, sys: 29.4 s, total: 1min 31s
Wall time: 1min 31s


,text,tokenized
25344,가임기 연력 여성의 흡연률 증가와 그 폐해 현대 여성은 다양한 사회활동에 참여하고 ...,"[가, 임기, 연력, 여성, 의, 흡연, 률, 증가, 와, 그, 폐해, 현대, 여성..."
25345,아토피에 유전적인 요인이 작용한다는 사실 알고 계시나요 유전적인 요인 뿐 아니라 아...,"[아토피, 에, 유전, 적, 인, 요인, 이, 작용, 한다는, 사실, 알, 고, 계..."
25346,1st 시기에 따라 필요한 영양소를 충분히 뱃속의 아이는 엄마가 먹는 것을 함께 먹...,"[1, st, 시기, 에, 따라, 필요, 한, 영양소, 를, 충분히, 뱃속, 의, ..."
25347,분만 여성만이 누릴 수 있는 특권 대부분의 여성들은 임신과 분만이라는 과정을 경험하...,"[분만, 여성, 만, 이, 누릴, 수, 있, 는, 특권, 대부분, 의, 여성, 들,..."
25348,최근 전자파가 건강상의 문제를 일으킬 수 있다는 견해가 많아지면서 임신부들 또한 가...,"[최근, 전자파, 가, 건강, 상, 의, 문제, 를, 일으킬, 수, 있, 다는, 견..."


In [10]:
df.isnull().sum()

text         0
tokenized    0
dtype: int64

## Train Word2Vec

#### WTF error
- df를 csv 형식으로 저장한 후 다시 로드하면 Word2Vec 학습에 인코딩 문제가 생긴다.
- 더 자세히 파고들어보니 list 형식의 column이 csv로 저장할 때 강제로 string 타입으로 형변환 됨.
- 이렇게 형변환 된 column을 Word2Vec 학습에 사용할 시 한글 텍스트가 깨지는 현상이 발생함.. 정확한 원인은 아직 모르겠다.

In [11]:
# df = pd.read_csv('text_for_word2vec.csv', encoding='utf-8')

그러므로 저장 & 로드 과정 생략하고 바로 말뭉치를 numpy ndarray형식으로 변환

In [12]:
corpus = np.array(df.tokenized)
type(corpus)

numpy.ndarray

In [13]:
%%time
model_all = Word2Vec(corpus, size=128, workers=4, sg=1)
model_all.save('data/tokmom_word2vec_all.model')

CPU times: user 6min 40s, sys: 801 ms, total: 6min 40s
Wall time: 1min 46s


#### 유사어 분석

In [14]:
model_all.wv.similar_by_word("놀이")

[('소꿉', 0.7444862127304077),
 ('공놀이', 0.7181103825569153),
 ('기차놀이', 0.7164962291717529),
 ('타악기', 0.700412392616272),
 ('보물찾기', 0.6973928213119507),
 ('실뜨기', 0.693272590637207),
 ('줄다리기', 0.6904656887054443),
 ('도미노', 0.6791845560073853),
 ('집짓기', 0.6781119108200073),
 ('숨바꼭질', 0.6736164093017578)]

In [15]:
model_all.wv.similar_by_word("아이")

[('아기', 0.720367968082428),
 ('어린아이', 0.6930141448974609),
 ('갓난아이', 0.6636514663696289),
 ('큰아이', 0.6503501534461975),
 ('친해진다', 0.633965790271759),
 ('빈이', 0.627231240272522),
 ('지낸다면', 0.6226592063903809),
 ('놀림거리', 0.620051383972168),
 ('엄마', 0.6195899844169617),
 ('들떠서', 0.6179345846176147)]

In [16]:
model_all.wv.similar_by_word("치아")

[('잇몸', 0.8262006640434265),
 ('영구치', 0.8096377849578857),
 ('충치', 0.7749719619750977),
 ('젖니', 0.7515202760696411),
 ('앞니', 0.7438544631004333),
 ('턱뼈', 0.7242212891578674),
 ('어금니', 0.7086251378059387),
 ('에나멜질', 0.7060268521308899),
 ('치태', 0.7001344561576843),
 ('법랑질', 0.6934532523155212)]

In [17]:
model_all.wv.similar_by_word("이빨")

[('혓바닥', 0.7661846876144409),
 ('핥', 0.7633767127990723),
 ('허옇', 0.7478742003440857),
 ('거뭇', 0.7460644245147705),
 ('빨아먹', 0.7341369986534119),
 ('거스러미', 0.7330491542816162),
 ('벅벅', 0.7242777347564697),
 ('밤색', 0.7240117788314819),
 ('럭비공', 0.7239671349525452),
 ('돌덩이', 0.7234801054000854)]

In [18]:
model_all.wv.similar_by_word("감기")

[('코감기', 0.8339973092079163),
 ('목감기', 0.8012640476226807),
 ('기침감기', 0.7839852571487427),
 ('환절기', 0.7697011828422546),
 ('독감', 0.7600629925727844),
 ('냉방병', 0.7590876221656799),
 ('호흡기', 0.749624490737915),
 ('비염', 0.7479081153869629),
 ('편도염', 0.7404163479804993),
 ('기관지염', 0.7400298118591309)]

In [19]:
model_all.wv.similar_by_word("열")

[('고열로', 0.587069034576416),
 ('오한', 0.5708358287811279),
 ('고열', 0.5692563056945801),
 ('열기', 0.5607172846794128),
 ('배탈', 0.5543304681777954),
 ('미열', 0.5440638661384583),
 ('불덩이', 0.5422656536102295),
 ('차가워서', 0.5371158123016357),
 ('차가워질', 0.5350601673126221),
 ('열작용', 0.5339616537094116)]

In [20]:
model_all.wv.similar_by_word("아파")

[('아프', 0.8142788410186768),
 ('아파서', 0.7715272903442383),
 ('다쳐', 0.758678138256073),
 ('다쳐서', 0.7369722723960876),
 ('아파도', 0.7318875789642334),
 ('칭얼댄다', 0.7285597324371338),
 ('아파하', 0.7255078554153442),
 ('토해서', 0.7212457656860352),
 ('거린다고', 0.7205687761306763),
 ('지끈지끈', 0.7157642841339111)]

In [21]:
model_all.wv.similar_by_word("아프")

[('아파', 0.8142788410186768),
 ('아파하', 0.770432710647583),
 ('결리', 0.7477769255638123),
 ('아파서', 0.747175931930542),
 ('지끈지끈', 0.7438468933105469),
 ('먹먹해', 0.7427558302879333),
 ('쑤시', 0.7415727376937866),
 ('어지럽', 0.7376511096954346),
 ('아픈데', 0.7361332774162292),
 ('어지러우', 0.7328156232833862)]

In [22]:
model_all.wv.similar_by_word("울음")

[('터뜨립니다', 0.69917893409729),
 ('터뜨릴', 0.6988707184791565),
 ('옹알이', 0.6852439045906067),
 ('터뜨린다', 0.6829092502593994),
 ('터뜨린', 0.6793863773345947),
 ('자지러질', 0.6625240445137024),
 ('괴성', 0.6569491624832153),
 ('어르', 0.6549452543258667),
 ('운다', 0.6500470638275146),
 ('자지러지', 0.6466145515441895)]

In [23]:
model_all.wv.similar_by_word("남아")

[('여아', 0.8464231491088867),
 ('정상아', 0.636469304561615),
 ('018', 0.6165052652359009),
 ('남자', 0.6160174012184143),
 ('클리토리스', 0.6009060144424438),
 ('경산부', 0.5969947576522827),
 ('남근기', 0.5943429470062256),
 ('구성비', 0.5778857469558716),
 ('여자', 0.5773581266403198),
 ('빨랐', 0.5763824582099915)]

In [24]:
model_all.wv.similar_by_word("여아")

[('남아', 0.8464231491088867),
 ('남자', 0.6111065149307251),
 ('여자', 0.5893025398254395),
 ('018', 0.5817461013793945),
 ('603', 0.5634723901748657),
 ('아든', 0.5592161417007446),
 ('617', 0.558576762676239),
 ('864', 0.5570540428161621),
 ('동월', 0.5523787140846252),
 ('클리토리스', 0.5461477041244507)]

---

## Only Noun

- 이번에는 말뭉치에서 명사만 추출하여 결과를 비교해보자
- 유사한 키워드 잡기를 목적으로 할 때는 명사만 추출하는 것이 더 적절한 듯

In [25]:
%%time
def extract_noun(text):
    tokenizer = Mecab()
    text = tokenizer.nouns(text)
#     text = ' '.join(text)
    return text
    
df['noun'] = df['text'].apply(extract_noun)
df = df.dropna()

CPU times: user 1min 3s, sys: 28.5 s, total: 1min 32s
Wall time: 1min 31s


In [99]:
noun_corpus = np.array(df.noun)

In [140]:
%%time
model_noun = Word2Vec(noun_corpus,
                      size=200,
                      window=2,
                      workers=4,
                      seed=1,
                      sg=1)
model_noun.save('data/tokmom_word2vec_only_noun.model')

CPU times: user 1min 51s, sys: 308 ms, total: 1min 52s
Wall time: 31.8 s


In [141]:
model_noun.wv.similar_by_word("육아")

[('양육', 0.5975416898727417),
 ('열혈', 0.574201762676239),
 ('재테크', 0.5728480219841003),
 ('워킹맘', 0.5722798705101013),
 ('바깥일', 0.5671976804733276),
 ('체험담', 0.5530815720558167),
 ('자처', 0.550591766834259),
 ('지원군', 0.5466318130493164),
 ('경험자', 0.5459321737289429),
 ('실속파', 0.544177770614624)]

In [142]:
model_noun.wv.similar_by_word("시댁")

[('시부모', 0.8119237422943115),
 ('시누이', 0.781395435333252),
 ('장모님', 0.771469235420227),
 ('친정어머니', 0.7712113857269287),
 ('처가', 0.7700274586677551),
 ('친정', 0.7647916078567505),
 ('시어머니', 0.7634627819061279),
 ('친가', 0.7548611164093018),
 ('식구', 0.7466983199119568),
 ('본가', 0.742530107498169)]

In [143]:
model_noun.wv.similar_by_word("놀이")

[('놀잇감', 0.6923749446868896),
 ('공놀이', 0.6575114727020264),
 ('소꿉', 0.6301233768463135),
 ('누가누', 0.6241747140884399),
 ('비석', 0.6136857271194458),
 ('기차놀이', 0.6128743290901184),
 ('지점토', 0.6123070120811462),
 ('도미노', 0.6084019541740417),
 ('타악기', 0.6039851903915405),
 ('점토', 0.603852391242981)]

In [144]:
model_noun.wv.similar_by_word("아이")

[('갓난아기', 0.6924479007720947),
 ('어린아이', 0.6844288110733032),
 ('입니다', 0.6837809085845947),
 ('밥투정', 0.682280421257019),
 ('말자', 0.6803841590881348),
 ('지레짐작', 0.6748100519180298),
 ('하민', 0.6726471781730652),
 ('무의식중', 0.6717808842658997),
 ('아기', 0.6660614013671875),
 ('갓난아이', 0.665270209312439)]

In [145]:
model_noun.wv.similar_by_word("치아")

[('잇몸', 0.7765349745750427),
 ('충치', 0.7297009229660034),
 ('영구치', 0.7276791334152222),
 ('젖니', 0.7213656902313232),
 ('어금니', 0.6917309761047363),
 ('법랑질', 0.6714276075363159),
 ('치조골', 0.6608656048774719),
 ('유치', 0.6586825847625732),
 ('치근', 0.6555143594741821),
 ('과잉치', 0.6543520092964172)]

In [146]:
model_noun.wv.similar_by_word("이빨")

[('혓바닥', 0.8135547637939453),
 ('거스러미', 0.7806691527366638),
 ('아랫입술', 0.7668402194976807),
 ('고무젖꼭지', 0.7647664546966553),
 ('덧', 0.7641605734825134),
 ('거립니다', 0.7641183137893677),
 ('풀칠', 0.7639509439468384),
 ('영인', 0.7630032300949097),
 ('밥풀', 0.7597187161445618),
 ('옷깃', 0.7564960718154907)]

In [147]:
model_noun.wv.similar_by_word("감기")

[('코감기', 0.7463159561157227),
 ('기침감기', 0.7336947917938232),
 ('목감기', 0.7328576445579529),
 ('냉방병', 0.7065818905830383),
 ('환절기', 0.7065317630767822),
 ('병치레', 0.705925703048706),
 ('편도염', 0.7054717540740967),
 ('기관지염', 0.6984241604804993),
 ('인후염', 0.6911981105804443),
 ('편도선염', 0.689525842666626)]

In [148]:
model_noun.wv.similar_by_word("열")

[('고열', 0.6074755787849426),
 ('고열로', 0.5936789512634277),
 ('오한', 0.5919020771980286),
 ('미열', 0.569339394569397),
 ('체기', 0.5578094720840454),
 ('불덩이', 0.5503739714622498),
 ('외감', 0.5490320324897766),
 ('사병', 0.5474042296409607),
 ('허열', 0.5456649661064148),
 ('설사약', 0.5436137914657593)]

In [149]:
model_noun.wv.similar_by_word("아파")

[('참기', 0.829912543296814),
 ('열꽃', 0.8245300054550171),
 ('무의식중', 0.8137537837028503),
 ('어린애', 0.8114954233169556),
 ('목도', 0.8076102137565613),
 ('자초', 0.8060405254364014),
 ('울상', 0.8059118390083313),
 ('얼떨결', 0.8056827783584595),
 ('마음고생', 0.8032673001289368),
 ('채찍', 0.8031811714172363)]

In [150]:
model_noun.wv.similar_by_word("울음")

[('지르기', 0.6773122549057007),
 ('비명', 0.6636983752250671),
 ('신음', 0.6632672548294067),
 ('놀람', 0.6615372896194458),
 ('아달라', 0.6602663993835449),
 ('세요', 0.641005277633667),
 ('옹알이', 0.6313501596450806),
 ('번역기', 0.6310714483261108),
 ('괴성', 0.630058765411377),
 ('음조', 0.6300169229507446)]

In [158]:
model_noun.wv.similar_by_word("남아")

[('여아', 0.8425937294960022),
 ('정상아', 0.6857849955558777),
 ('생시', 0.6791380047798157),
 ('확연', 0.6736302375793457),
 ('체구', 0.6668820381164551),
 ('자폐아', 0.6614252924919128),
 ('남근기', 0.6579033136367798),
 ('아일', 0.6538131833076477),
 ('항문기', 0.650348424911499),
 ('출생체중', 0.6464788317680359)]

In [160]:
model_noun.wv.similar_by_word("여아")

[('남아', 0.8425937294960022),
 ('확연', 0.6213060021400452),
 ('체구', 0.6208465099334717),
 ('남근기', 0.606103777885437),
 ('항문기', 0.5981905460357666),
 ('남학생', 0.5954718589782715),
 ('페르모', 0.5921984314918518),
 ('성별', 0.5917976498603821),
 ('생시', 0.5914918184280396),
 ('정상아', 0.5911825299263)]

In [153]:
model_noun.wv.similar_by_word("여자")

[('남자', 0.814863383769989),
 ('여아', 0.5723517537117004),
 ('남아', 0.5700061917304993),
 ('남학생', 0.5446677207946777),
 ('금성', 0.5405828952789307),
 ('조신', 0.5388423204421997),
 ('외톨', 0.5321864485740662),
 ('여탕', 0.5302890539169312),
 ('남탕', 0.5256863832473755),
 ('아이든', 0.5252732038497925)]

In [154]:
model_noun.wv.similar_by_word("남자")

[('여자', 0.814863383769989),
 ('금성', 0.6181306838989258),
 ('남아', 0.6027320623397827),
 ('조신', 0.5889028310775757),
 ('아이든', 0.5859947204589844),
 ('얌전', 0.583115816116333),
 ('여탕', 0.5814687013626099),
 ('여아', 0.5749674439430237),
 ('외톨', 0.5742570757865906),
 ('남학생', 0.5722742676734924)]

In [155]:
model_noun.wv.similar_by_word("유모차")

[('디럭스', 0.7416549921035767),
 ('절충', 0.6769294738769531),
 ('잉글', 0.6337890625),
 ('휴대용', 0.6321314573287964),
 ('타보', 0.6249380707740784),
 ('퀴니', 0.6235971450805664),
 ('페도라', 0.6125385761260986),
 ('범용', 0.6007155179977417),
 ('맥클라렌', 0.5981160402297974),
 ('카시트', 0.5922831296920776)]

In [156]:
model_noun.wv.similar_by_word("카시트")

[('컨버터블', 0.6521413326263428),
 ('안전벨트', 0.634675145149231),
 ('브라이', 0.6275078058242798),
 ('콩코드', 0.6192984580993652),
 ('장착', 0.6172091960906982),
 ('뒷좌석', 0.6171060800552368),
 ('부스터', 0.6134747266769409),
 ('페도라', 0.6109854578971863),
 ('캐리어', 0.6088404655456543),
 ('웨건', 0.6054836511611938)]

---

## Noun + Verb + Adj

- 마지막으로 말뭉치에서 명사, 동사, 형용사까지 추출한 결과를 확인

In [69]:
%%time
def extract_keywords(text):
    tokenizer = Mecab()
    pos_list = tokenizer.pos(text)
    keywords_list = list(filter(lambda w: 'NNG' in w or \
                                           'NNP' in w or \
                                           'VA' in w or \
                                           'VV' in w , pos_list))
    keywords = list(map(lambda w: w[0], keywords_list))
    return keywords

df['keywords'] = df['text'].apply(extract_keywords)
df = df.dropna()

CPU times: user 1min 4s, sys: 26.3 s, total: 1min 30s
Wall time: 1min 30s


In [72]:
%%time
model_keywords = Word2Vec(keywords_corpus, size=100, workers=4, sg=1)
model_keywords.save('data/tokmom_word2vec_keywords.model')

CPU times: user 3min 8s, sys: 227 ms, total: 3min 8s
Wall time: 51 s


In [73]:
model_keywords.wv.similar_by_word("육아")

[('워킹맘', 0.7055859565734863),
 ('양육', 0.6735218167304993),
 ('독박', 0.6716964840888977),
 ('열혈', 0.6540733575820923),
 ('지원군', 0.6444226503372192),
 ('박찬희', 0.6383612155914307),
 ('아서', 0.6352624893188477),
 ('장세희', 0.6350053548812866),
 ('휴직', 0.6331909894943237),
 ('살림', 0.6330448389053345)]

In [74]:
model_keywords.wv.similar_by_word("시댁")

[('친정', 0.8407643437385559),
 ('본가', 0.8244757056236267),
 ('시부모', 0.8029784560203552),
 ('처가', 0.7992140054702759),
 ('시어머니', 0.7828656435012817),
 ('식구', 0.7771471738815308),
 ('친정어머니', 0.7748686671257019),
 ('대소사', 0.7744762301445007),
 ('시누이', 0.7651897668838501),
 ('시아버님', 0.7620663046836853)]

In [75]:
model_keywords.wv.similar_by_word("놀이")

[('놀잇감', 0.7531840801239014),
 ('소꿉', 0.7382116317749023),
 ('재미있', 0.7330796718597412),
 ('공놀이', 0.7296560406684875),
 ('놀', 0.7147164344787598),
 ('보물찾기', 0.7133928537368774),
 ('집짓기', 0.7097890377044678),
 ('무너뜨리기', 0.708432674407959),
 ('샌드백', 0.7035702466964722),
 ('음률', 0.6929599046707153)]

In [76]:
model_keywords.wv.similar_by_word("아이")

[('어린아이', 0.7821304798126221),
 ('엄마', 0.7402162551879883),
 ('하나', 0.7188742160797119),
 ('동질', 0.7185378074645996),
 ('돌쟁이', 0.7179890871047974),
 ('아이라', 0.7173634767532349),
 ('말자', 0.7159072160720825),
 ('뒤쳐지', 0.7158945202827454),
 ('살찌우', 0.7146617770195007),
 ('생각', 0.7131155133247375)]

In [77]:
model_keywords.wv.similar_by_word("치아")

[('잇몸', 0.8642995953559875),
 ('영구치', 0.8340755701065063),
 ('충치', 0.8255764245986938),
 ('젖니', 0.8106557130813599),
 ('앞니', 0.8081176280975342),
 ('어금니', 0.7645289897918701),
 ('법랑질', 0.761794924736023),
 ('유치', 0.753669798374176),
 ('윗니', 0.7530847787857056),
 ('우식증', 0.7441324591636658)]

In [78]:
model_keywords.wv.similar_by_word("이빨")

[('골짜기', 0.7855191230773926),
 ('혓바닥', 0.7829934358596802),
 ('영인', 0.7778735160827637),
 ('갉', 0.7768616080284119),
 ('근질거리', 0.7685598731040955),
 ('오물거리', 0.7606532573699951),
 ('역겹', 0.7602251768112183),
 ('부둥켜안', 0.7521517276763916),
 ('거스러미', 0.7517303228378296),
 ('까맣', 0.7506961226463318)]

In [79]:
model_keywords.wv.similar_by_word("감기")

[('목감기', 0.8507241010665894),
 ('코감기', 0.831957995891571),
 ('기침감기', 0.8000133633613586),
 ('환절기', 0.7884349822998047),
 ('편도염', 0.7822831869125366),
 ('기관지염', 0.7788915634155273),
 ('편도선염', 0.7728430032730103),
 ('냉방병', 0.7679721117019653),
 ('호흡기', 0.7670795917510986),
 ('열나', 0.757851779460907)]

In [80]:
model_keywords.wv.similar_by_word("열")

[('열기', 0.6857141852378845),
 ('오한', 0.6692895293235779),
 ('고열로', 0.6670310497283936),
 ('외감', 0.6613254547119141),
 ('불덩이', 0.6497596502304077),
 ('식히', 0.6434990167617798),
 ('허열', 0.6433795690536499),
 ('몸살감기', 0.6416375637054443),
 ('미열', 0.6338551640510559),
 ('열나', 0.6324325799942017)]

In [81]:
model_keywords.wv.similar_by_word("아파")

[('어린애', 0.8438390493392944),
 ('아파하', 0.8381340503692627),
 ('참기', 0.829542875289917),
 ('쓰라리', 0.814190685749054),
 ('아프', 0.8095778226852417),
 ('생채기', 0.8086069822311401),
 ('놀래', 0.7983182072639465),
 ('나지', 0.7944431304931641),
 ('겁나', 0.7896738648414612),
 ('불덩이', 0.7878525853157043)]

In [82]:
model_keywords.wv.similar_by_word("울음")

[('우', 0.8077030181884766),
 ('울', 0.7894013524055481),
 ('흐느끼', 0.7190532684326172),
 ('자지러지', 0.7188630104064941),
 ('괴성', 0.7169601917266846),
 ('칭얼거리', 0.7118950486183167),
 ('비명', 0.7046586275100708),
 ('지르', 0.703108012676239),
 ('어르', 0.6996307373046875),
 ('신음', 0.6962636709213257)]

In [83]:
model_keywords.wv.similar_by_word("남아", topn=20)

[('여아', 0.8582215309143066),
 ('정상아', 0.6962495446205139),
 ('성징', 0.6893209218978882),
 ('기라', 0.6852234601974487),
 ('색맹', 0.684944212436676),
 ('아도', 0.678516149520874),
 ('클리토리스', 0.6607379913330078),
 ('정상인', 0.6599722504615784),
 ('위수', 0.654329240322113),
 ('극소', 0.6498882174491882),
 ('증후', 0.6492331624031067),
 ('생시', 0.6480215787887573),
 ('변성기', 0.6475589275360107),
 ('두드러지', 0.6446006894111633),
 ('조숙', 0.6428589820861816),
 ('확연', 0.6371349096298218),
 ('보인자', 0.6352572441101074),
 ('정류', 0.6328153610229492),
 ('출생체중', 0.6306383609771729),
 ('남자', 0.6286284327507019)]

In [84]:
model_keywords.wv.similar_by_word("여아", topn=20)

[('남아', 0.8582215309143066),
 ('성징', 0.6465380191802979),
 ('여자', 0.625258207321167),
 ('기라', 0.6163921356201172),
 ('남자', 0.6067968010902405),
 ('가슴둘레', 0.5990121960639954),
 ('위수', 0.5850467681884766),
 ('성별', 0.5789700746536255),
 ('성비', 0.5730557441711426),
 ('아도', 0.5671846866607666),
 ('클리토리스', 0.5667832493782043),
 ('여탕', 0.5654733180999756),
 ('성조숙', 0.5639233589172363),
 ('생시', 0.5600930452346802),
 ('남근', 0.5589829683303833),
 ('정상아', 0.5585267543792725),
 ('확연', 0.5564078688621521),
 ('변성기', 0.5538173913955688),
 ('아이든', 0.5514933466911316),
 ('성차', 0.5510605573654175)]

In [85]:
model_keywords.wv.similar_by_word("여자", topn=20)

[('남자', 0.9276505708694458),
 ('조신', 0.7118227481842041),
 ('여탕', 0.6932961940765381),
 ('성차', 0.6865707635879517),
 ('남탕', 0.67728191614151),
 ('금성', 0.6694213151931763),
 ('엘렉트라', 0.6342495679855347),
 ('아이든', 0.632871150970459),
 ('성별', 0.6269031167030334),
 ('여아', 0.625258207321167),
 ('족쇄', 0.6226381659507751),
 ('남아', 0.6187950372695923),
 ('일운', 0.6127138137817383),
 ('남학생', 0.6084304451942444),
 ('바람둥이', 0.6025266647338867),
 ('시집살이', 0.6019200682640076),
 ('에너자이저', 0.6012762784957886),
 ('야동', 0.601057231426239),
 ('우스갯소리', 0.5986984968185425),
 ('알파걸', 0.598076581954956)]

In [86]:
model_keywords.wv.similar_by_word("남자", topn=20)

[('여자', 0.9276505708694458),
 ('조신', 0.7535786032676697),
 ('여탕', 0.7398945689201355),
 ('성차', 0.7190895080566406),
 ('남탕', 0.7090216875076294),
 ('금성', 0.7009105682373047),
 ('얌전', 0.6762772798538208),
 ('성별', 0.6730945110321045),
 ('족쇄', 0.6637382507324219),
 ('아이든', 0.6455618143081665),
 ('일운', 0.6429769992828369),
 ('알파걸', 0.6423494219779968),
 ('엘렉트라', 0.6410560011863708),
 ('유다이', 0.6398763656616211),
 ('들추', 0.6361589431762695),
 ('숫기', 0.6341906189918518),
 ('야동', 0.6329840421676636),
 ('처갓집', 0.6317377090454102),
 ('우스갯소리', 0.6308197975158691),
 ('무뚝뚝', 0.6302056312561035)]

In [87]:
model_keywords.wv.similar_by_word("유모차")

[('디럭스', 0.814447283744812),
 ('절충', 0.7793101072311401),
 ('잉글', 0.757431149482727),
 ('레시', 0.7374228239059448),
 ('핸들링', 0.7270081043243408),
 ('휴대용', 0.7255921363830566),
 ('카시트', 0.7218056917190552),
 ('타보', 0.7169990539550781),
 ('맥클라렌', 0.7101566791534424),
 ('삼륜', 0.7096701860427856)]

In [88]:
model_keywords.wv.similar_by_word("카시트")

[('브라이', 0.7454673051834106),
 ('안전벨트', 0.7432515621185303),
 ('부스터', 0.7399038076400757),
 ('장착', 0.7338041067123413),
 ('컨버터블', 0.7242361307144165),
 ('유모차', 0.7218056321144104),
 ('뒤보', 0.7155474424362183),
 ('콩코드', 0.7092009782791138),
 ('순성', 0.7090553045272827),
 ('페도라', 0.7064604759216309)]

---

### Volume of Corpus 

In [89]:
df[['text']].to_csv('data/text_for_word2vec.csv', index=False, encoding='utf-8')

In [90]:
pd.read_csv('data/text_for_word2vec.csv')

,text
0,6살 남아 입니다 회사 복직 후 3살부터 주중에는 외할머니댁에서 2살 4살 터울의 ...
1,요즘 들어 아빠가 아이에게 다가가려고 하면 엄마한테 갈 거야 하면서 엄마를 유독 찾...
2,둘째가 태어나면 첫째가 힘들 거라는 건 예상하고 있었지만 툭하면 삐치고 울고 좀처럼...
3,20개월 남아가 있고 임신 8개월차로 두 아기의 엄마입니다 첫 아이는 잠자리 독립을...
4,14개월 남자 아기인데 심하게 깨무는 버릇이 있어요 전에는 졸릴 때 주로 물곤 했는...
...,...
25344,가임기 연력 여성의 흡연률 증가와 그 폐해 현대 여성은 다양한 사회활동에 참여하고 ...
25345,아토피에 유전적인 요인이 작용한다는 사실 알고 계시나요 유전적인 요인 뿐 아니라 아...
25346,1st 시기에 따라 필요한 영양소를 충분히 뱃속의 아이는 엄마가 먹는 것을 함께 먹...
25347,분만 여성만이 누릴 수 있는 특권 대부분의 여성들은 임신과 분만이라는 과정을 경험하...


---

## Doc2Vec

In [91]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [92]:
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(df['noun'])]

In [93]:
%%time
doc2vec_model = Doc2Vec(documents, 
                        vector_size=100, 
                        window=5, 
                        min_count=5, 
                        workers=4)

model.save('data/tokmom_doc2vec_only_noun.model')

NameError: name 'model' is not defined

In [94]:
len(doc2vec_model.docvecs)

25349

In [95]:
df['text'][0]

'6살 남아 입니다 회사 복직 후 3살부터 주중에는 외할머니댁에서 2살 4살 터울의 사촌 형 누나와 함께 자랐어요 금요일 저녁에 데려와 주말에는 엄마 아빠와 생활하고 있구요 7세가 되는 내년이라도 아이를 데려오자는 아이 아빠의 의견과 유치원과 태권도 등 패턴이 적응되었고 형 누나들과 더 자랄 수 있게 초등학교 입학시기에 데리고 가는게 낫지 않겠냐는 외할머니 의견이 다릅니다 6세가 된 아이의 교육과 생활습관 형성이 이제는 할머니댁에서는 어려울 것 같다는 생각이 들고 하루하루 생각이 크고 자라는 아이를 주말에만 보는 저 또한 놀랄 때가 많거든요 하루라도 빨리 데려와야 할까요 초등학교 입학시기에 데려오는 편이 아이에게 좋을까요 매우 어려운 질문입니다 먼저 아이가 현재 잘 적응하고 있고 주변 양육환경 역시 아이가 성장하기에 별다른 문제가 없다 하니 고민이 더욱 크시리라 생각됩니다 우선 되어야하는 것은 아이의 의견입니다 6살이면 이러한 문제에 대하여 스스로 생각하고 의견을 전달할 수 있으므로 먼저 아이의 의견을 들어 주시기 바랍니다 아이는 부모님과 함께 사는 것이 좋습니다 그러나 부모님께서 두분 모두 직장에 나가시니 유치원 하원 후 아이를 돌보아 주실 분이 현재 함께 생활하고 계시는 할머님이시면 재고의 여지 없이 데려 오시는 것을 권유 드립니다 그러나 하원 후 할머님의 돌봄이 여의치 못할 경우 돌보아 줄 대상이 당분간 할머님댁에서 시간제로나마 아이를 돌보아 주어 아이와의 상호작용을 충분히 가진 후에 아이가 집으로 왔을 때 유치원 하원 후에 계속 돌보아 줄 수 있으면 좋습니다 이와 같이 먼저 아이가 집으로 온 후 부모님께서 퇴근하시기 전 까지 돌보아 줄 사람을 알아보시고 데려 오시는 것이 좋다는 생각이 듭니다'

In [96]:
print("Input Document", df['text'][100], sep="\n")

similar_docs = doc2vec_model.docvecs.most_similar(100)

print("Most similar Documents")
for idx, similarity in similar_docs[:3]:
    print("idx:", idx, "similarity:", np.round(similarity, 3))
    print(df['text'][idx])

Input Document
젖병을 떼야 하는 이유로는 첫째 우식증을 예방하고 둘째 젖병 무는 어린 아기의 습관을 서서히 고쳐 나가기 위함 입니다 지금은 단지 습관에 의해 젖병을 물고 있을 뿐이므로 음식은 숟가락으로 먹이고 우유는 컵으로 마시게 하면서 젖병을 뗄 수 있도록 해야 합니다 잠잘 때 젖병을 물고 자는 것은 무의식적인 행동입니다 그러나 너무 친숙해진 습관을 갑자기 금지시킨다면 아기 심리에 큰 충격을 줄 수도 있으므로 서서히 젖병 속의 우유를 희석시키는 방법을 사용합니다 예를 들어 평상시에 우유를 200 mL 먹였다면 우유 180 mL에 물 20 mL를 섞어서 젖병에 넣어줍니다 점차 물의 농도를 높여가다가 아기가 이를 느끼고 부정적인 반응을 보일 때는 잠시 정지하였다가 다시 희석의 강도를 높여봅니다 그러다 보면 아기는 점차 희석된 우유에 흥미를 잃어 스스로 젖병을 내던지게 될 것입니다 아기에 따라 스스로 정서적 안정을 얻는 습관을 하나씩 가지고 있습니다 아기 때의 이불을 늘 잡고 있다거나 엄마의 머리카락을 잡아야 잠을 잔다거나 하는 습관은 고치기가 쉽지 않은 일입니다 젖병도 그런 습관의 일종일 수 있습니다 젖병을 물고 있는 것이 정서적인 이유라면 젖병 떼기를 강요하지 않습니다 하루 2 3번은 젖병을 물고 있게 하다가 점차 아기가 성장하면서 젖병 무는 것이 자신에게는 어울리지 않는다는 것을 엄마가 끊임없이 설득하고 아이 스스로도 그렇게 느껴 자연스레 젖병을 멀리하게 합니다 가능한 컵이나 빨대컵을 활용하고 노리개 젖꼭지 등을 대체하여 사용해 볼 수도 있습니다
Most similar Documents
idx: 21274 similarity: 0.617
자다가 깨서 울고 젖병을 뺏어도 울고 엄마가 잠시라도 곁에 없어도 우는 아이 갑자기 울보가 돼버린 아이가 걱정되겠지만 아이 정서에 문제가 있어서가 아니라 이 시기 아이가 잘 자라고 있다는 신호로 보면 된다 무엇보다 엄마와의 애착관계가 형성되는 시기로 이때 정서적으로 안정감을 느낀 아이는 훗날 독립적이고 사회성 좋

In [97]:
df['tokenized_doc'] = df['noun'].apply(lambda x: doc2vec_model.infer_vector(x).astype(float))

In [98]:
df

,text,tokenized,noun,keywords,tokenized_doc
0,6살 남아 입니다 회사 복직 후 3살부터 주중에는 외할머니댁에서 2살 4살 터울의 ...,"[6, 살, 남, 아, 입, 니, 다, 회사, 복직, 후, 3, 살, 부터, 주, ...","[살, 회사, 복직, 후, 살, 주, 중, 외할머니, 댁, 살, 살, 터울, 사촌,...","[남, 입, 회사, 복직, 후, 주, 외할머니, 댁, 터울, 사촌, 형, 누나, 금...","[0.18320201337337494, -0.14881069958209991, -0..."
1,요즘 들어 아빠가 아이에게 다가가려고 하면 엄마한테 갈 거야 하면서 엄마를 유독 찾...,"[요즘, 들, 어, 아빠, 가, 아이, 에게, 다가가, 려고, 하, 면, 엄마, 한...","[아빠, 아이, 엄마, 거, 엄마, 전, 정도, 엄마, 아빠, 거부, 엄마, 눈, ...","[들, 아빠, 아이, 다가가, 하, 엄마, 하, 엄마, 찾, 전, 정도, 엄마, 매...","[0.05332105606794357, 0.05352022126317024, -0...."
2,둘째가 태어나면 첫째가 힘들 거라는 건 예상하고 있었지만 툭하면 삐치고 울고 좀처럼...,"[둘째, 가, 태어나, 면, 첫째, 가, 힘들, 거, 라는, 건, 예상, 하, 고,...","[둘째, 첫째, 거, 건, 예상, 때, 엄마, 껌, 딱지, 첫째, 외출, 때, 제외...","[태어나, 예상, 삐치, 울, 서럽, 웁, 때, 엄마, 껌, 딱지, 외출, 때, 제...","[0.1522061675786972, 0.09916505217552185, -0.0..."
3,20개월 남아가 있고 임신 8개월차로 두 아기의 엄마입니다 첫 아이는 잠자리 독립을...,"[20, 개월, 남아, 가, 있, 고, 임신, 8, 개월, 차, 로, 두, 아기, ...","[개월, 남아, 임신, 개월, 차, 아기, 엄마, 아이, 잠자리, 독립, 생후, 일...","[남아, 있, 임신, 차, 아기, 엄마, 아이, 잠자리, 독립, 생후, 전, 재우,...","[0.11638510227203369, 0.10280551016330719, 0.0..."
4,14개월 남자 아기인데 심하게 깨무는 버릇이 있어요 전에는 졸릴 때 주로 물곤 했는...,"[14, 개월, 남자, 아기, 인데, 심하, 게, 깨무, 는, 버릇, 이, 있, 어...","[개월, 남자, 아기, 버릇, 전, 때, 이제, 자기, 지대, 때, 떼, 상대방, ...","[남자, 아기, 심하, 깨무, 버릇, 있, 전, 때, 물, 이제, 지대, 때, 떼,...","[0.027467967942357063, -0.18142782151699066, 0..."
...,...,...,...,...,...
25344,가임기 연력 여성의 흡연률 증가와 그 폐해 현대 여성은 다양한 사회활동에 참여하고 ...,"[가, 임기, 연력, 여성, 의, 흡연, 률, 증가, 와, 그, 폐해, 현대, 여성...","[임기, 연력, 여성, 흡연, 증가, 폐해, 현대, 여성, 사회, 활동, 참여, 경...","[임기, 연력, 여성, 흡연, 증가, 폐해, 현대, 여성, 사회, 활동, 참여, 경...","[-0.4393390715122223, -0.8597395420074463, -0...."
25345,아토피에 유전적인 요인이 작용한다는 사실 알고 계시나요 유전적인 요인 뿐 아니라 아...,"[아토피, 에, 유전, 적, 인, 요인, 이, 작용, 한다는, 사실, 알, 고, 계...","[아토피, 유전, 요인, 작용, 사실, 유전, 요인, 뿐, 아토피, 유발, 수, 하...","[아토피, 유전, 요인, 작용, 사실, 알, 유전, 요인, 아토피, 유발, 있, 중...","[0.20945513248443604, -0.03948991745710373, 0...."
25346,1st 시기에 따라 필요한 영양소를 충분히 뱃속의 아이는 엄마가 먹는 것을 함께 먹...,"[1, st, 시기, 에, 따라, 필요, 한, 영양소, 를, 충분히, 뱃속, 의, ...","[시기, 필요, 영양소, 뱃속, 아이, 엄마, 것, 균형, 영양, 음식, 섭취, 태...","[시기, 필요, 영양소, 뱃속, 아이, 엄마, 먹, 먹, 균형, 잡히, 영양, 있,...","[-0.01232465635985136, -0.4101027548313141, -0..."
25347,분만 여성만이 누릴 수 있는 특권 대부분의 여성들은 임신과 분만이라는 과정을 경험하...,"[분만, 여성, 만, 이, 누릴, 수, 있, 는, 특권, 대부분, 의, 여성, 들,...","[분만, 여성, 수, 특권, 대부분, 여성, 임신, 분만, 과정, 경험, 삶, 의미...","[분만, 여성, 있, 특권, 대부분, 여성, 임신, 분만, 과정, 경험, 삶, 의미...","[-0.20566150546073914, -0.2236362248659134, -0..."
